In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', -1)

In [2]:
data = pd.read_csv('pristavki.csv', header=None, names=['text'])

In [3]:
from yargy import Parser, rule, or_
from yargy.predicates import in_, in_caseless
from yargy.tokenizer import MorphTokenizer
from yargy.pipelines import morph_pipeline, caseless_pipeline
from yargy.interpretation import fact
from IPython.display import display

In [4]:
game_for_pristavka = fact('game', ['name', 'number'])

In [10]:
fifa = rule(
            morph_pipeline(['фифа', 'Фифв', 'FIFA', 'мартол комбат', 'Mortal combat', 'гта', 'GTA', 
                            'асассин', 'Assassins Creed', 'видмак','Call of Duty', 'Call of Duty:', 'Killzone',
                            'Battlefield', 'Battlefield:', 'Farcry', 'Need For Speed', 'Need For Speed:',
                            'Один из нас', 'Borderlends', 'Borderlands', 'Brothers in arms',
                            'DESTINY','Mass Effect', 'Super Street Fighter', 'Dead space', 'The army of two',
                            'Dead to rights', 'Праздник спорта', 'Granturismo', 'Звезды Playstation',
                            'Dualshock', 'мортал комбат', 'nfs', 'god of war', 'фар край', 'кол оф дьюти',
                            'Infamous second son', 'Minecraft', 'Driveclub', 'UFS', 'Метро', 'Fallout',
                            'Ведьмак', 'Batman', 'Wolfenstein', 'UFC', 'Bloodborn', 'Watch dogs', 'MGS',
                            'The last of us', 'dualshok', 'Kill Zone', 'Орден', 'Second son',
                            'Garden warfare', 'PES', 'NHL', 'NBA', 'Mortal Kombat', 'little big planet',
                            'Uncharted', 'Fight night', 'Michael Jackson', 'Max Payne', 'Max. Payne',
                            'Дурная репутация', 'Smack down vs raw', 'far cry', 'Gran Turismo', 
                            'Rayman', 'Skylanders', 'unit', 'Beyond', 'Beyond:', 'Collapse', 'Warcraft',
                            'Stalker', 'Warrior', 'Мафия', 'Alone dark', 'Mercenaries', 'Sniper', 'Тамплиеры',
                            'Квейк', 'Dewil may cry', 'Devil may cry', 'Quake', 'Witcher', 'Worms',
                            'Hospital tycoon', 'Flashpoint', 'Mortal', 'Dirt', 'Tennis', 'Gold of wars',
                            'Grand Turismo', 'Brother in arms', 'Brother in arms:', 'Анчартед', 'анчартид',
                            'Horizon zero dawn', 'Crash', 'Horizont zero dawn', 'Resident evil'
                            ]).interpretation(game_for_pristavka.name),
            morph_pipeline(['15', '16', '13', '17', '2014', '11', 'vs D.C.', '5', '4', 'V', '3', 'Ghosts', 
                            '4 Modern Warfare', 'MW3', 'Modern Warfare', 'Black Ops', 'Bad company', '2', 
                           'Bad company 2', 'Hardline', 'Shift', 'Rivals', 'Битва сильнейших', 'X', 'Х',
                            '2016', 'unity', '400', '600', '9', 'undisputed', '2009', 'round 4', 'champion',
                            'Legends', 'Origins', 'Two souls', 'Earned In Blood', 'Road To Hill 30', '7'
                           ]).interpretation(game_for_pristavka.number).optional()
            )

In [11]:
GAME = (fifa).interpretation(game_for_pristavka)

parser = Parser(GAME)

In [12]:
matches = []

for sent in data.text[:1000]:
    for match in parser.findall(sent):
        matches.append(match.fact)

In [13]:
for m in matches:
    print(m.name, m.number)

фифа 15
мартол комбат 9
гта 5
асассин 4
Фифв 13
гта 4
видмак 3
Mortal combat vs D.C.
FIFA 16
GTA 5
FIFA 2014
Call of Duty Ghosts
Call of Duty 4 Modern warfare
Call of Duty MW3
Killzone 3
Battlefield 3
Farcry 3
Assassins Creed 4
Need For Speed Shift
Один из нас None
Fifa 11
Fifa 15
Borderlends None
Brothers in arms None
UFS 3
праздник спорта None
Одни из нас None
gta 5
DESTINY None
Battlefield 4
GTA V
Battlefield: Bad company 2
Battlefield: Hardline
Call Of Duty: Black Ops
Call Of Duty: Modern Warfare
Need For Speed: Rivals
Mass Effect 3
Super Street Fighter 5
Fifa None
Одни из нас None
Dead space 3
The army of two None
Dead to rights None
Праздник спорта 2
Battlefield 4
Granturismo 3
Звезды Playstation Битва сильнейших
GTA 5
dualshock 4
мортал комбат Х
nfs None
god of war None
Battlefield 4
Nfs 2016
фар край None
кол оф дьюти None
Battlefield 4
Infamous second son None
Minecraft None
Call of Duty None
GTA V
Driveclub None
UFS 2
Одни из нас None
Метро None
Fallout None
Ведьмак None
Batm

In [9]:
len(matches)

716

Показались интересными следующие особенности:

1. Капитализация или ее отсутствие в исходном правиле не влияет на результат работы парсера — при FIFA в правиле находятся и fifa, и FIFA, и Fifa
2. В случае, если в исходном тексте между названием игры и ее номером нет пробела, парсер все равно правильно отрабатывает — например, в оригинале написано фифа15, а спаршивается фифа 15
3. Если указать часть сущности optional(), то при ненахождении мэтча она заменяется на None, а не остается пустой
4. Судя по всему, парсер избирательно исправляет некоторые незначительные ошибки — например, при наличии в правиле варианта «Один из нас» он выделяет не только такое название, но и «Одни из нас». При этом, например, при наличии в правиле FIFA ошибочный вариант FAFA не выделяется